<a href="https://colab.research.google.com/github/AchrafAsh/gnn-receptive-fields/blob/main/02_experiment_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODOS
- [ ] add wandb.log to log things like total parameters, configs, etc
- [ ] use an MLP for READOUT instead of another GCN layer (makes more sense for our method)

# Install, import and log in

In [1]:
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
try:
    os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
except:
    pass
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [2]:
# import everything
import math
import random
import copy
import time
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

from tqdm.notebook import tqdm
from typing import Dict, List, Tuple
from torch_geometric.utils import to_dense_adj, dense_to_sparse, add_self_loops

from torch_geometric.nn import GCNConv, MessagePassing, Sequential
from torch_geometric.utils import degree, to_dense_adj

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%matplotlib inline
sns.set_style('darkgrid')

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Import data

In [4]:
%%capture
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py

from data import load_dataset

path = osp.join(os.getcwd(), 'data')
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0] # only graph of the dataset
# G.edge_index.shape  # [2, 10556] → 10_556 edges

In [54]:
def get_k_neighbors(k: int):
    """Returns the l-hop neighbors for l between 1 (the adjacency matrix) and k (given depth)

    Args:
        - adj: dense adjacency matrix
        - k (int): size of the maximum neighborhood
    """
    
    output = [G.edge_index]
    dense_adj = to_dense_adj(G.edge_index).squeeze(0)
    dense_nebs = [dense_adj.clone()]
    adj_pow = dense_adj.clone()

    for l in tqdm(range(1, k)):
        adj_pow = torch.mm(dense_adj, adj_pow)
        k_neb = torch.where(
            torch.where(adj_pow > 0, 1, 0) - sum(dense_nebs) > 0,
            1,
            0
        )
        dense_nebs.append(k_neb)
        output.append(dense_to_sparse(k_neb)[0])
    
    return output

# Define experiment pipeline

In [64]:
def model_pipeline(hyperparameters):
    with wandb.init(project='gnn-receptive-fields', entity='achraf', config=hyperparameters):
            config = wandb.config # to make sure we use what is saved in wandb

            # create the model
            model, criterion, optimizer = make(config)
            
            # compute different depth edge_index
            all_edge_index = get_k_neighbors(config.num_layers)

            # train the model
            train(model, all_edge_index, criterion, optimizer, config)

            # Save the model in the exchangeable ONNX format → not working... yet
            # torch.onnx.export(model, G, "model.onnx")
            # wandb.save("model.onnx")

    return model

### Build the model

In [21]:
# Parameter initialization

def xavier(tensor):
    """Initialize weight matrix with Xavier distribution

    Args:
        tensor (tensor): weigh matrix
    Return:
        tensor - weight matrix initialized
    """
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-2)))
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    """Initialize bias vector with all zeros

    Args:
        tensor (tensor): bias vector
    
    Return
        tensor - bias vector initialized with zeros
    """
    if tensor is not None:
        tensor.data.fill_(0)

In [7]:
class HopGCNConv(MessagePassing):
    def __init__(self, num_features:int, in_channels:int, out_channels:int, k:int):
        super(HopGCNConv, self).__init__(aggr='add')  # "Add" aggregation
        self.k = k
        self.lin_input = torch.nn.Linear(num_features, out_channels)
        self.lin_hidden = torch.nn.Linear(in_channels, out_channels)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        xavier(self.lin_input.weight)
        zeros(self.lin_input.bias)
        
        xavier(self.lin_hidden.weight)
        zeros(self.lin_hidden.bias)

    def forward(self, x, h, edge_index):
        # x is the input features and has shape [N, num_features]
        # h is the hidden state and has shape [N, in_channels]
        # edge_index has shape [2, E] , E being the number of edges

        # step 1: linearly transform node feature matrices
        x = self.lin_input(x)
        h = self.lin_hidden(h)

        # step 3-5: start propagating messages
        return self.propagate(edge_index[self.k], x=x, h=h)

    def message(self, x_j, h_i, edge_index, size):
        # x_j is the input features of the neighbors and has shape [E, out_channels] (has already been multiplied by the weight matrix)

        # step 3: normalize node features
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        out = norm.view(-1, 1) * x_j

        return out + h_i
        # add the hidden state of the target node
        # norm = deg_inv_sqrt[row] * deg_inv_sqrt[row]
        
        # return out + norm.view(-1, 1) * h_i

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels] is the output of self.message()

        # step 5: return new node embeddings
        return aggr_out

In [40]:
class KHopNet(torch.nn.Module):
    def __init__(self, num_layers:int, hidden_dim:int, num_features:int, num_classes:int, dropout:float=0.5, invert:bool=False):
        super().__init__()
        self.conv_layers = self.create_layers(num_layers=num_layers, 
                                              num_features=num_features,
                                              num_classes=num_classes,
                                              hidden_dim=hidden_dim,
                                              dropout=dropout,
                                              invert=invert)
        
        self.fc = torch.nn.Linear(in_features=hidden_dim,
                                  out_features=num_classes)        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)


    def create_layers(self, num_layers:int, num_features:int,
                      hidden_dim:int, dropout:float):
        layers = []

        # first layer
        layers += [
                (HopGCNConv(num_features=num_features, in_channels=num_features, out_channels=hidden_dim, k=0), "x, x, edge_index -> h"),
                (torch.nn.ReLU(), "h -> h"),
                (torch.nn.Dropout(p=dropout), "h -> h")
        ]

        for k in range(1, num_layers):
            layers += [
                (HopGCNConv(num_features=num_features, in_channels=hidden_dim, out_channels=hidden_dim, k=k), "x, h, edge_index -> h"),
                (torch.nn.ReLU(), "h -> h"),
                (torch.nn.Dropout(p=dropout), "h -> h")
            ]

        return Sequential("x, edge_index", layers)


    def reset_parameters(self):
        self.conv_layers.reset_parameters()


    def forward(self, x, edge_index):
        h = self.conv_layers(x, edge_index)
        out = self.log_softmax(self.fc(h))
        return h, out

In [12]:
def make(config):
    # Make the model
    model = KHopNet(num_layers=config.num_layers, hidden_dim=config.hidden_dim,
                    num_features=cora_dataset.num_features,
                    num_classes=cora_dataset.num_classes,
                    dropout=config.dropout).to(device)

    # Make the loss and optimizer
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=config.learning_rate,
                                 weight_decay=config.weight_decay)
    
    return model, criterion, optimizer

## Train, test functions

In [ ]:
def train(model, all_edge_index, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    for epoch in tqdm(range(config.epochs)):
        loss = train_step(model, all_edge_index, optimizer, criterion)
        
        # test the model
        test(model, all_edge_index)


def train_step(model, all_edge_index, optimizer, criterion):
    """Performs one training step
    """
    model.train()
    
    # Forward pass
    _, out = model(G.x, all_edge_index)
    loss = criterion(out[G.train_mask], G.y[G.train_mask])
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [60]:
def test(model, all_edge_index, metrics=[]):
    """
    Metrics is a list of tuple ('metric_name', metric_func) where the metric 
    function takes the last representation matrix and returns a scalar.
    """

    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        h, logits = model(G.x, all_edge_index)

    outs = {}
    for (name, metric) in metrics:
        outs[name] = metric(h)

    for key in ['train', 'val', 'test']:
        mask = G[f'{key}_mask']
        loss = F.nll_loss(logits[mask], G.y[mask]).item()
        pred = logits[mask].max(1)[1]
        acc = pred.eq(G.y[mask]).sum().item() / mask.sum().item()

        outs[f'{key}_loss'] = loss
        outs[f'{key}_acc'] = acc
    
    wandb.log(outs)
    
    return outs

In [67]:
config = dict(
    num_layers=8,
    hidden_dim=16,
    epochs=1000,
    learning_rate=0.001,
    weight_decay=1e-5,
    dropout=0.5)

# Run experiments

In [ ]:
model = model_pipeline(config)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# Look at the weights distributions
# histograms of the weights of one layer

for param in model.parameters():
    # find which layer it belongs to
    print(param.)

    # look at the values
    print(param.data)
